In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev

In [2]:
experiments = {
    'BrushlessMotor': {
        '790':
        {
            'conv_end_to_end': 'experiments/BrushlessMotor-sync_head_conv-False-True-0.5',
            'conv_pretrained': 'experiments/BrushlessMotor-sync_head_conv-True-False-0.5',
            'spline': 'experiments/BrushlessMotor-resample_spline-False-False-0.5'
        },
        '1537':
        {
            'conv_end_to_end': 'experiments/BrushlessMotor-sync_head_conv-False-True-1',
            'conv_pretrained': 'experiments/BrushlessMotor-sync_head_conv-True-False-1',
            'spline': 'experiments/BrushlessMotor-resample_spline-False-False-1'
        }
    }
}
AUC = {}
for machine, machine_experiments in experiments.items():
    AUC[machine] = {}
    for window_length, model_paths in machine_experiments.items():
        AUC[machine][window_length] = {}
        for model, path in model_paths.items():
            df = pd.read_csv(
                f'{path}/AUC_results.csv')
            df.columns = ["SensorName", "Overall", "Source", "Target"]
            AUC[machine][window_length][model] = df

In [ ]:
from matplotlib.ticker import MultipleLocator

# Adjust these parameters based on your needs
WIDTH = 14  # Width of the figure
HEIGHT_PER_PLOT = 3  # Height per subplot

for machine, machine_dict in AUC.items():
    window_lengths = list(machine_dict.keys())
    num_windows = len(window_lengths)
    
    # Create figure with vertical subplots for different window lengths
    fig, axes = plt.subplots(num_windows, 1, figsize=(WIDTH, HEIGHT_PER_PLOT * num_windows), 
                             sharex=True, squeeze=False)
    axes = axes.flatten()  # Flatten in case we have a single subplot
    
    # Get all models (assuming same models across window lengths)
    first_window = next(iter(machine_dict.values()))
    models = list(first_window.keys())
    
    for ax, window_length in zip(axes, window_lengths):
        model_dict = machine_dict[window_length]
        first_model = models[0]
        df_first = model_dict[first_model]
        sensors = df_first['SensorName'].values
        x = np.arange(len(sensors))
        
        # Calculate bar positions
        bar_width = 0.8 / len(models)
        offsets = np.arange(len(models)) * bar_width - (len(models) - 1) * bar_width / 2
        
        # Plot each model's Overall AUC
        for model_idx, model in enumerate(models):
            df = model_dict[model]
            ax.bar(x + offsets[model_idx], df['Overall'], 
                   width=bar_width, label=model)
        
        # Configure axes
        ax.set_xticks(x)
        ax.set_xticklabels(sensors, rotation=45, ha='right')
        ax.set_title(f'Window {window_length}')
        ax.grid(True)
        ax.yaxis.set_major_locator(MultipleLocator(0.1))
        ax.set_ylim(0, 1)
        
        # Add y-label to all plots
        ax.set_ylabel('Overall AUC')

    # Add common elements
    plt.suptitle(f'{machine} - Overall AUC Comparison', y=1.02)
    
    # Add a single legend for the entire figure
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(0.99, 0.99))
    
    plt.tight_layout()
    #plt.savefig(f"{machine}_Overall_AUC_comparison.png", bbox_inches='tight')
    #plt.close(fig)